In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Modules

In [14]:
from subprocess import PIPE, run, STDOUT
#from os import environ

#environ['TOMA'] = 'BBBBB'
print(run('module list', stdout=PIPE, stderr=STDOUT, shell=True).stdout.decode('utf-8'))

/bin/sh: module: command not found



# Parameters

In [21]:
from operator import mul
from functools import reduce
from datetime import datetime

params = {
    # Executable
   'Executable path': '/home/teonnik/code/asynchronator/build/example',
   'Executable name': 'laplace3d_jacobi',
    
    # Linear system
   'Matrix': 'laplace3d',
   'Domain partitions': [[0, 50],[0, 50],[0, 50]],
   'Initial guess': 'zero',
   'Exact solution': 'white-noise sine',

    # Solver
   'Iterant': 'jacobi',
   'Steps': 10,
   'Max iterations': 100000,
   "Tolerance": 0.01,
   "Timeout [min]": 1    
}

# Device
params['Device'] = 'julia'
params['Processes per node'] = 1
params['Threads per process'] = 256
params['Timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M")
params['Processes grid'] = [len(x) - 1 for x in params['Domain partitions']]
params['Nodes'] = reduce(mul, params['Processes grid'], 1)

params

{'Device': 'julia',
 'Domain partitions': [[0, 50], [0, 50], [0, 50]],
 'Exact solution': 'white-noise sine',
 'Executable name': 'laplace3d_jacobi',
 'Executable path': '/home/teonnik/code/asynchronator/build/example',
 'Initial guess': 'zero',
 'Iterant': 'jacobi',
 'Matrix': 'laplace3d',
 'Max iterations': 100000,
 'Nodes': 1,
 'Processes grid': [1, 1, 1],
 'Processes per node': 1,
 'Steps': 10,
 'Threads per process': 256,
 'Timeout [min]': 1,
 'Timestamp': '2018-04-16 14:22',
 'Tolerance': 0.01}

# Command

In [22]:
job = ('sbatch '
       '--job-name={executable} '
       '--nodes={nodes} '
       '--ntasks-per-node={procs} '
       '--cpus-per-task={threads} '
       '--time={timeout} '
       '--mail-user=teodor.nikolov22@gmail.com '
       '--mail-type=FAIL '
       '--error=err.txt '
       '--output=out.txt '
       '--wrap=\"srun {execDir}/{executable}\"').format(
    executable=params['Executable name'],
    execDir=params['Executable path'],
    procs=params['Processes per node'],
    nodes=params['Nodes'],
    threads=params['Threads per process'],
    timeout='{:0>2d}:{:0>2d}:00'.format(params['Timeout [min]'] // 60, 
                                        params['Timeout [min]'] % 60)
)
job

'sbatch --job-name=laplace3d_jacobi --nodes=1 --ntasks-per-node=1 --cpus-per-task=256 --time=00:01:00 --mail-user=teodor.nikolov22@gmail.com --mail-type=FAIL --error=err.txt --output=out.txt --wrap="srun /home/teonnik/code/asynchronator/build/example/laplace3d_jacobi"'

# Submit

In [23]:
from subprocess import run
from pathlib import Path
from json import dump
from datetime import datetime

# Create a run directory
runDir = Path('{}_{}'.format(params['Executable name'], 
                             datetime.now().strftime("%Y-%m-%d_%H-%M-%S")))
runDir.mkdir(exist_ok=False)

# Write setup file
with (runDir / 'setup.json').open('w') as setupF: 
    dump(params, setupF)

# Run the job
run(job, cwd=runDir, shell=True)

CompletedProcess(args='sbatch --job-name=laplace3d_jacobi --nodes=1 --ntasks-per-node=1 --cpus-per-task=256 --time=00:01:00 --mail-user=teodor.nikolov22@gmail.com --mail-type=FAIL --error=err.txt --output=out.txt --wrap="srun /home/teonnik/code/asynchronator/build/example/laplace3d_jacobi"', returncode=127)

# Postprocess

In [ ]:
import pandas as pd
from pathlib import Path
from json import load
from os import stat

# Check for errors
if stat(runDir / 'err.txt').st_size != 0:
    raise ValueError('The error file is not empty! Check err.txt!')
if stat(runDir / 'out.txt').st_size != 0:
    raise ValueError('Something was written to standard output! Check out.txt!')

# Read files into dictionary
data = []
for recPath in simulationPath.glob('*.json'):
    with recPath.open('r') as recF: 
        data.append(load(recF))

# Preview table
df = pd.DataFrame(data)
df.head()

# Save to .csv

In [ ]:
df.to_csv('{}.csv'.format(runDir.name))